# Create an InterActive Graph of NTRL Lab Analysis Results

In [5]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [6]:
#fcn dfn
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(colname):
    dfcn = df_main[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        df_main[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return df_main
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df
#---------------------------------------


In [11]:
def lab_analysis_auto_update():    
    # generates a dataframe for each column in the excel file
    wb1 = openpyxl.load_workbook(r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\6) June _2020.xlsx', data_only=True)
    #####This block of code appends the Analysis Results from 
    NTRL_lab = wb1['104PU01']
    MS_lab = wb1['106TK01']
    #update for the targetmonth
    currentmonthyr = datetime.date(2020,6,1)

    ntrl_Pb_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'E', '104_Pb')
    ntrl_Zn_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'F', '104_Zn')
    ntrl_Cu_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'G', '104_Cu')
    ntrl_Ni_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'H', '104_Ni')
    ntrl_Co_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'I', '104_Co')
    ntrl_Mn_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'K', '104_Mn')
    ntrl_Cr_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'L', '104_Cr')
    ntrl_Ca_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'M', '104_Ca')
    ntrl_Si_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'N', '104_Si')
    ntrl_Al_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'O', '104_Al')
    ntrl_Mg_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'P', '104_Mg')
    ntrl_Fe2_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'R', '104_Fe2')

    ntrl_pH_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'AI', '104_pH')
    ntrl_ORP_df = generate_df_from_analysis(NTRL_lab, 'B', 'C', 'AJ', '104_ORP')

    ntrl_PbZn = ntrl_Pb_df.join(ntrl_Zn_df, how='outer')
    ntrl_CuNi = ntrl_Cu_df.join(ntrl_Ni_df, how='outer')
    ntrl_Co = ntrl_Co_df
    ntrl_MnCr = ntrl_Mn_df.join(ntrl_Cr_df, how='outer')
    ntrl_CaSi = ntrl_Ca_df.join(ntrl_Si_df, how='outer')
    ntrl_AlMg = ntrl_Al_df.join(ntrl_Mg_df, how='outer')
    ntrl_pHORP = ntrl_pH_df.join(ntrl_ORP_df, how='outer')

    #--------------
    ntrl_PbZnCuNi = ntrl_PbZn.join(ntrl_CuNi, how='outer')
    ntrl_CoFeMnCr = ntrl_Co.join(ntrl_MnCr, how='outer')
    ntrl_CaSiAlMg = ntrl_CaSi.join(ntrl_AlMg, how='outer')
    ntrl_Fe2pHORP = ntrl_Fe2_df.join(ntrl_pHORP, how='outer')

    ntrl_PbZnCuNiCoFeMnCr = ntrl_PbZnCuNi.join(ntrl_CoFeMnCr, how='outer')
    ntrl_CaSiAlMgFe2Fe3pHORP = ntrl_CaSiAlMg.join(ntrl_Fe2pHORP, how='outer')

    ntrl_PbZnCuNiCoFeMnCrCaSiAlMgFe2Fe3pHORP = ntrl_PbZnCuNiCoFeMnCr.join(ntrl_CaSiAlMgFe2Fe3pHORP, how='outer')
    #--------------
    df_main = ntrl_PbZnCuNiCoFeMnCrCaSiAlMgFe2Fe3pHORP

    #fill the nan values with the average of surrounding values
    for i in range(0, len(df_main.columns)):
        #print(i, df_main.columns[i])
        ave_filler(df_main.columns[i])
        df_main = df_main.fillna(method='bfill')

    df_main2 = df_main[['104_Pb', '104_Zn', '104_Cu', '104_Ni', '104_Co', '104_Mn',
            '104_Cr', '104_Ca', '104_Si', '104_Al', '104_Mg', '104_Fe2', '104_pH',
            '104_ORP',]]
    df_main2 = df_main2.dropna(how='any')
    print('Done', datetime.datetime.now())

    y_Pb_ntrl = df_main['104_Pb']
    y_Zn_ntrl = df_main['104_Zn']
    y_Cu_ntrl = df_main['104_Cu']
    y_Ni_ntrl = df_main['104_Ni']
    y_Co_ntrl = df_main['104_Co']
    y_Mn_ntrl = df_main['104_Mn']
    y_Cr_ntrl = df_main['104_Cr']
    y_Ca_ntrl = df_main['104_Ca']
    y_Si_ntrl = df_main['104_Si']
    y_Al_ntrl = df_main['104_Al']
    y_Mg_ntrl = df_main['104_Mg']
    y_Fe2_ntrl = df_main['104_Fe2']
    y_pH_ntrl = df_main['104_pH']
    y_ORP_ntrl = df_main['104_ORP']

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(go.Scatter(x=y_Zn_ntrl.index, y=y_Zn_ntrl, name="NTRL_Zn"), secondary_y=False,)
    fig.add_trace(go.Scatter(x=y_Pb_ntrl.index, y=y_Pb_ntrl, name="NTRL_Pb"), secondary_y=False,)
    fig.add_trace(go.Scatter(x=y_Cu_ntrl.index, y=y_Cu_ntrl, name="NTRL_Cu"), secondary_y=False,)
    fig.add_trace(go.Scatter(x=y_Ni_ntrl.index, y=y_Ni_ntrl, name="NTRL_Ni"), secondary_y=True,)
    fig.add_trace(go.Scatter(x=y_Co_ntrl.index, y=y_Co_ntrl, name="NTRL_Co"), secondary_y=False,)
    fig.add_trace(go.Scatter(x=y_Mn_ntrl.index, y=y_Mn_ntrl, name="NTRL_Mn"), secondary_y=True,)
    fig.add_trace(go.Scatter(x=y_Cr_ntrl.index, y=y_Cr_ntrl, name="NTRL_Cr"), secondary_y=False,)
    fig.add_trace(go.Scatter(x=y_Ca_ntrl.index, y=y_Ca_ntrl, name="NTRL_Ca"), secondary_y=False,)
    fig.add_trace(go.Scatter(x=y_Si_ntrl.index, y=y_Si_ntrl, name="NTRL_Si"), secondary_y=False,)
    fig.add_trace(go.Scatter(x=y_Al_ntrl.index, y=y_Al_ntrl, name="NTRL_Al"), secondary_y=True,)
    fig.add_trace(go.Scatter(x=y_Mg_ntrl.index, y=y_Mg_ntrl, name="NTRL_Mg"), secondary_y=True,)
    fig.add_trace(go.Scatter(x=y_Fe2_ntrl.index, y=y_Fe2_ntrl, name="NTRL_Fe2"), secondary_y=False,)
    fig.add_trace(go.Scatter(x=y_pH_ntrl.index, y=y_pH_ntrl, name="NTRL_pH"), secondary_y=True,)
    fig.add_trace(go.Scatter(x=y_ORP_ntrl.index, y=y_ORP_ntrl, name="NTRL_ORP"), secondary_y=True,)

    #title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
    t = datetime.datetime.now()
    title_plot = "<b>NTRL Laboratory Analysis Results June 2020</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'

    # Add figure title
    fig.update_layout(title_text=title_plot)

    # Set x-axis title
    fig.update_xaxes(title_text="DateTime")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Pb, Zn , Cu, Co, Cr, Ca, Si, Fe2</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Ni, Mn, Al, Mg, pH, ORP</b>", secondary_y=True)
    #save the figure to html
    fig.write_html(r'C:\Users\v.t.flores\Documents\LaboratoryResults.html')
    fig.write_html(r'\\thgobna002\userdata\THPAL\05 Production\150 DCS\Miscellaneous\LaboratoryResults.html')
    print('Update log:', datetime.datetime.now())

In [ ]:
#this code updates the analysis file every hour for 30 days
for i in range(0, 30):
    for j in range(0, 24):
        lab_analysis_auto_update()
        time.sleep(3600)

Done 2020-06-02 08:22:20.578525
